In [2]:
from datasets import load_dataset
dataset = load_dataset("rajpurkar/squad")
    # dataset = load_dataset("quac")

    # train and validation
train = dataset['train']
validation = dataset['validation']

In [10]:
import getopt
import sys

from kwQnA._exportPairs import exportToJSON
from kwQnA._getentitypair import GetEntity
from kwQnA._graph import GraphEnt
from kwQnA._qna import QuestionAnswer

import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm






class Main:
    """docstring for Main."""

    def __init__(self):
        super(Main, self).__init__()
        self.qna = QuestionAnswer()
        self.getEntity = GetEntity()
        self.export = exportToJSON()
        self.graph = GraphEnt()

    def main(self, argv):
        inputfile = ''
        inputQue = ''
        try:
            opts, args = getopt.getopt(argv, "hi:q:g:s:", ["ifile=", "question=","showGraph=","showEntities="])
            if opts == [] and args == []:
                print("ERROR")
                print("Help:")
                print("python init.py -i <TextFileName> -q <Question> -s <show Ent>")
        except getopt.GetoptError as err:
            sys.exit(2)

        for opt, arg in opts:
            showGraph , showEntities= "f", "f"
            if opt == '-h':
                print ('test.py -i <inputfile> -q <question> -g <y or n> -s <Show Entities>')
                sys.exit()
            elif opt in ("-i", "--ifile"):
                inputfile = arg
            elif opt in ("-q", "--question"):
                inputQue = arg
            elif opt in ("-g", "--showGraph"):
                showGraph = arg
            elif opt in ("-s", "--showEntities"):
                showEntities = arg
            else:
                assert False, "unhandled option"

        return inputfile, inputQue, showGraph, showEntities


def QuestionStartsWith_Accuracy(initialize, dataset, startsWith):
    correct = 0
    total = 0
    errors = []
    startsWith = startsWith.lower()
    for item in tqdm(dataset):
        try:
            context = item['context'].lower()
            context = re.sub(' +', ' ', context)
            
            question = item['question'].lower()
            question = question.replace("manys", "many")
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0].lower()
            title = item['title'].lower()
    
            # check if question starts with startsWith
            if question.startswith(startsWith):
                total += 1
    
                # if (total < 3):
                if (total not in (2442, 2574, 2772, 3202, 3828, 4360)):       # when
                    continue
                question = question.replace("news papers", "newspapers")

                # if (total in (4414, 905)):        # who
                # if (total in (18, 16153, 16155, 16158)):       # what
                # if (total in (2442, 2574, 2772, 3202, 3828, 4360)):       # when
                # if (total in (3, 906, 1420, 1656, 1659, 1661, 1797, 1798, 1802, 1803, 1953, 1954, 1955, 2915)):       # where
                # if (total in (3039, 3900)):       # how many
                    # print("Debug")
                    
                refined_text = initialize.getEntity.preprocess_context(context, title)
                dataEntities, numberOfPairs = initialize.getEntity.get_entity(refined_text)
                if dataEntities:
                    initialize.export.dumpdata(dataEntities[0])

                    outputAnswer = initialize.qna.findanswer(question, numberOfPairs)
                    if outputAnswer in answer or answer in outputAnswer:
                        correct += 1

                    # else:
                    #     print("Wrong:", total)
                    #     print("Question: ", question)
                    #     print("Answer: ", answer)
                    #     print("Output: ", outputAnswer)
                    #     print("\n\n")

                
                if total % 2000 == 0 or (total % 100 == 0 and startsWith != "what"):
                    print(f"Accuracy of {startsWith}: {100*correct/total}")
                    print(f"Correct: {correct}, out of {total}")
                # else:
                #     print("Not Applicable - mfeesh entities")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Accuracy of {startsWith}: {100*correct/total}")
        print(f"Correct: {correct}, out of {total}")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    return correct, total, errors


if __name__ == "__main__":
    initialize = Main()

    # dataset = load_dataset("rajpurkar/squad")
    # # dataset = load_dataset("quac")

    # # train and validation
    # train = dataset['train']
    # validation = dataset['validation']


    x = QuestionStartsWith_Accuracy(initialize, train, "when")
    



100%|██████████| 87599/87599 [00:10<00:00, 8029.64it/s]

Accuracy of when: 0.0
Correct: 0, out of 5453
Errors:  []


In [ ]:
print(x)

# count = 0
    # titles = set()
    # max_context = 0
    # max_contexts = 0
    # prev_title = ""
    # for item in train:
    #     if prev_title != item['title']:
    #         context = item['context']
    #     elif item['context'] not in context:
    #         context += " " + item['context']
    #     max_context = max(max_context, len(item['context']))
    #     max_contexts = max(max_contexts, len(context))
    #     if (max_contexts == 92637):
    #         print(context)
    #         break
    #     prev_title = item['title']
    #     titles.add(item['title'])
    #     context = item['context'].lower()
    #     question = item['question'].lower()
    #     answer = item['answers']['text'][0].lower()
    #     if (answer in context):
    #         count += 1
    # total = len(train)
    # print(total)
    # print(count)
    # print(len(titles))
    # print(max_context)
    # print(max_contexts)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
# dataset = load_dataset("quac")

# train and validation
train = dataset['train']
validation = dataset['validation']

# save them to csv
# train.to_csv('dataset/train.csv')
# validation.to_csv('dataset/validation.csv')
